In [1]:
from igdb_puller import pull_games_and_dependents, pull_tables_as_globals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path
import re

RuntimeError: Missing TWITCH_CLIENT_ID/TWITCH_CLIENT_SECRET. Set env vars or add them to a .env file.

In [2]:
pull_games_and_dependents(max_games=500, include_second_order=True)

NameError: name 'pull_games_and_dependents' is not defined

In [3]:
for k, v in sorted((k, v) for k, v in globals().items() if k.startswith('df_') and isinstance(v, pd.DataFrame)):
    print(k, v.shape)

df_age_ratings (1438, 9)
df_artworks (858, 10)
df_covers (500, 9)
df_external_games (1904, 15)
df_franchises (81, 8)
df_game_localizations (282, 8)
df_game_modes (5, 7)
df_game_time_to_beats (219, 9)
df_game_videos (907, 5)
df_games (500, 56)
df_genres (21, 7)
df_involved_companies (1566, 10)
df_keywords (1862, 7)
df_language_supports (2541, 7)
df_multiplayer_modes (139, 14)
df_platforms (80, 17)
df_player_perspectives (6, 7)
df_popularity_primitives (2439, 9)
df_release_dates (2679, 16)
df_screenshots (2910, 9)
df_websites (2483, 6)
